## **PIPELINE**

En esta última práctica, veremos cómo realizar la puesta en producción de un modelo de Machine Learning.

El pipeline es una forma de codificar y automatizar el workflow necesario para producir un modelo de Machine Learning. Los pipelines en ML constan de varios pasos secuenciales que hacen de todo, desde la extracción y el preprocesamiento de datos hasta el entrenamiento y la implementación de modelos.

En definitiva, nos ayuda a automatizar un flujo de trabajo de Machine Learning.

<img src = "./_src/assets/pipeline_2.jpg" height = 350>

Uno de sus grandes beneficios y aplicabilidad reside en la escalabilidad. A su vez, destaca entre sus ventajas que simplifica la aplicación de cross-validation y optimización de hiperparámetros.

Con este enfoque, podemos *empaquetar* todas las acciones que van desde el preprocesamiento de los datos hasta las predicciones del modelo en un único objeto.

La librería `Scikit-learn` incluye una clase, denominada precisamente **pipeline**, que está diseñada para poder manipular y aplicar una serie de transformaciones de datos seguida de la aplicación de un modelo de ML. En otras palabras, con esta clase podemos instanciar un objeto que incluya todo el flujo del preprocesamiento de datos -desde la limpieza, normalización, escalado, transformación, reducción de dimensionalidad hasta ingeniería de features- y el modelo predictivo con el cual haremos las inferencias.

También Scikit-learn nos brinda la posibilidad de hacer nuestro pipeline con la clase **make_pipeline**. Veamos brevemente en qué se diferencia una de la otra.

|Pipeline|Make_pipeline|
|-|-|
|Hay que especificar todos los pasos secuenciales en una lista|No hay que indicar un nombre para cada paso|
|Cada paso es una tupla con el nombre del paso y su nombre oficial en Scikit-learn|Se pueden encadenar todos los pasos usando sus nombres oficiales en Scikit-learn|
|Ejemplo: `a`|Ejemplo: `b`|

```python

'a' Pipeline([('imputer', SimpleImputer(strategy='median')),('scaler', StandardScaler())])

```

```python

'b' make_pipeline(SimpleImputer(strategy='median'), StandardScaler())

```

- - -

## `Práctica`

Para la realización de esta práctica construiremos tres pipelines, cada uno con un modelo diferente. 

Primero haremos algunos pasos del preprocesamiento de datos, luego adaptaremos nuestros modelos. Para finalizar, compararemos la performance de los modelos y buscaremos identificar el mejor de ellos. Una vez obtenido, lo guardamos en un archivo.

In [1]:
# Cargamos los datos con los que vamos a estar trabajando

from sklearn.datasets import load_iris

iris = load_iris()

In [2]:
# Separamos los datos en train y test

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.2, random_state=42)

A continuación, crearemos los tres pipelines. Los modelos que usaremos serán: regresión logística, support vector machine y árbol de decisión. Respecto al preprocesamiento, aplicaremos un escalado de datos con `Standard Scaler` y haremos reducción de dimensionalidad con `PCA`.

In [3]:
# Importamos las librerías necesarias

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn import tree

In [4]:
# Creamos el primer pipeline

pipe_lr = Pipeline([('scl', StandardScaler()), # estandarización de datos 
			('pca', PCA(n_components=2)), # reducción de dimensionalidad con PCA previa a la regresión logística
			('clf', LogisticRegression(random_state=42))]) # pipeline de regresión logística con PCA y estandarización de datos 

In [5]:
# Creamos el segundo pipeline

pipe_svm = Pipeline([('scl', StandardScaler()), # estandarización de datos 
			('pca', PCA(n_components=2)), # reducción de dimensionalidad con PCA previa a SVM
			('clf', svm.SVC(random_state=42))]) # pipeline con SVM como clasificador final y PCA como reductor de dimensionalidad previo al clasificador final (SVM) 
			


In [6]:
# Creamos el tercer pipeline

pipe_dt = Pipeline([('scl', StandardScaler()), # estandarización de datos
			('pca', PCA(n_components=2)), # reductor de dimensionalidad previo al clasificador final (árbol de decisión) 
			('clf', tree.DecisionTreeClassifier(random_state=42))]) # Creamos el pipeline con los parámetros que queramos para cada paso del mismo (en este caso, el clasificador es un árbol de decisión) 

In [7]:
# Los guardamos en una lista

pipelines = [pipe_lr, pipe_svm, pipe_dt]

In [8]:
# Hacemos un diccionario para fines organizativos

pipe_dict = {0: 'Regresión Logística', 1: 'SVM', 2: 'Árbol de decisión'} # Diccionario para fines organizativos de los resultados de los modelos de clasificación binaria (0,1)

In [9]:
# Entrenamos

for pipe in pipelines:
	pipe.fit(X_train, y_train)

In [10]:
# Evaluamos

for idx, val in enumerate(pipelines):
	print(idx)
	print('%s pipeline accuracy en test: %.3f' % (pipe_dict[idx], val.score(X_test, y_test))) # Imprimimos la precisión de cada modelo de clasificación binaria (0,1) en test 

0
Regresión Logística pipeline accuracy en test: 0.900
1
SVM pipeline accuracy en test: 0.900
2
Árbol de decisión pipeline accuracy en test: 0.867


In [11]:
# Identificamos el mejor modelo para el set de testeo

best_acc = 0.0
best_clf = 0
best_pipe = ''
for idx, val in enumerate(pipelines):
	if val.score(X_test, y_test) > best_acc:
		best_acc = val.score(X_test, y_test)
		best_pipe = val
		best_clf = idx
print('Modelo con el mejor accuracy: %s' % pipe_dict[best_clf])

Modelo con el mejor accuracy: Regresión Logística


In [12]:
# Guardamos el pipeline en un archivo
# import pickle
import joblib #pickle es alternativa

joblib.dump(best_pipe, 'Mejor_pipeline.pkl', compress=1) # Guardamos el pipeline en un archivo para su uso posterior.
print('Pipeline de %s guardado a archivo' % pipe_dict[best_clf]) # Imprimimos el nombre del pipeline guardado en el archivo para fines organizativos 

loaded_model = joblib.load('Mejor_pipeline.pkl') # Cargamos el pipeline guardado en un archivo

Pipeline de Regresión Logística guardado a archivo


In [13]:
mejor_clasificador = joblib.load('Mejor_pipeline.pkl') # Cargamos el pipeline que guardamos en el archivo Mejor_pipeline.pkl

In [14]:
# guardar con pickle
# pickle.dump(best_pipe, open('Mejor_pipeline.pkl', 'wb'))

# Cargamos el pipeline

# import pickle
# loaded_model = pickle.load(open('Mejor_pipeline.pkl', 'rb'))

# Cargamos nuestro modelo

In [15]:
best_model = joblib.load('Mejor_pipeline.pkl') # Cargamos el pipeline guardado en un archivo 

X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.2, random_state=100) # Separamos los datos en train y test donde vamos a predecir con el modelo guardado en el archivo


- - -

In [16]:
print('Accuracy del modelo cargado: %.3f' % best_model.score(X_train, y_train))

Accuracy del modelo cargado: 0.908


In [17]:
best_model.score(X_test, y_test)
print('Accuracy del modelo guardado: %.3f' % best_model.score(X_test, y_test))

Accuracy del modelo guardado: 0.933
